<a href="https://colab.research.google.com/github/poojaanbu0/genai-multidoc-retrieval/blob/main/Multidoc_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exp 4: Building a Multi-Document Agent

In [ ]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
papers = [
    "docs/metagpt.pdf",
    "docs/longlora.pdf",
    "docs/selfrag.pdf",
]

In [ ]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: docs/metagpt.pdf
Getting tools for paper: docs/longlora.pdf
Getting tools for paper: docs/selfrag.pdf


In [ ]:
initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
len(initial_tools)

6

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools,
    llm=llm,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
# The agent can then choose the appropriate tools for each steps
response = agent.query(
    "Tell me about the evaluation dataset used in LongLoRA, "
    "and then tell me about the evaluation results"
)

Added user message to memory: Tell me about the evaluation dataset used in LongLoRA, and then tell me about the evaluation results
=== Calling Function ===
Calling function: summary_tool_longlora with args: {"input": "evaluation dataset"}
=== Function Output ===
The evaluation dataset used in the provided context includes the RedPajama dataset, PG19 validation split, book corpus dataset PG19, cleaned Arxiv Math proof-pile dataset, LongBench, LEval, and LongAlpaca-12k.
=== Calling Function ===
Calling function: summary_tool_longlora with args: {"input": "evaluation results"}
=== Function Output ===
The evaluation results across various experiments and comparisons demonstrate the effectiveness of different methods in extending context windows, achieving comparable performance to full fine-tuning while reducing computational costs, improving efficiency, maintaining performance in long context adaptation, and showcasing state-of-the-art performance in handling long-context tasks. The resul

In [ ]:
# we can chat/query about the LLM's reasoning process
response = agent.chat(
    "Tell me about the tools you leverage to answer the above question."
)

Added user message to memory: Tell me about the tools you leverage to answer the above question.
=== LLM Response ===
To provide information on the evaluation dataset and evaluation results in LongLoRA, I utilized the following tools:

1. **LongLoRA Summary Tool**: This tool was used to extract a summary of the evaluation dataset used in LongLoRA, including details about the datasets involved in the evaluation process.

2. **LongLoRA Summary Tool**: I also used this tool to generate a summary of the evaluation results in LongLoRA, which includes insights into the performance of different methods, comparisons, and configurations in various experiments.

These tools helped me efficiently gather and present the relevant information about the evaluation dataset and evaluation results in LongLoRA.


In [ ]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: docs/metagpt.pdf
Getting tools for paper: docs/longlora.pdf
Getting tools for paper: docs/selfrag.pdf


In [ ]:
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [ ]:
# define an "object" index over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [ ]:
# define the "retriever" over the index with specified retrieval method
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [ ]:
tools = obj_retriever.retrieve(
    "Tell me about the eval dataset used in MetaGPT and longLora"
)

In [ ]:
# check for the top 3 tools selected
tools[0].metadata

ToolMetadata(description='Use ONLY IF you want to get a holistic summary of MetaGPT. Do NOT use if you have specific questions over MetaGPT.', name='summary_tool_longlora', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [ ]:
# Define the agentWorker and agentRunner
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm,
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Tell me about the evaluation dataset used "
    "in MetaGPT and compare it against longLora"
)
print(str(response))

Added user message to memory: Tell me about the evaluation dataset used in MetaGPT and compare it against longLora
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation dataset used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT are HumanEval, MBPP, and SoftwareDev.
=== Calling Function ===
Calling function: summary_tool_longlora with args: {"input": "evaluation dataset used in longLora"}
=== Function Output ===
The evaluation dataset used in LongLoRA is the PG19 validation set from Rae et al., 2020.
=== LLM Response ===
The evaluation datasets used in MetaGPT are HumanEval, MBPP, and SoftwareDev. On the other hand, the evaluation dataset used in longLora is the PG19 validation set from Rae et al., 2020.
assistant: The evaluation datasets used in MetaGPT are HumanEval, MBPP, and SoftwareDev. On the other hand, the evaluation dataset used in longLora is the PG19 validation set from Rae et al., 2020.
